# Transition path theory

TPT

https://www.nature.com/articles/srep17986

In [ ]:
start = (7, 2)

In [ ]:
import sktime
import numpy as np
sim = sktime.data.drunkards_walk(bar_location=(0, 0), home_location=(9, 9))
walk = sim.walk(start=start, n_steps=250, seed=17)

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import FancyArrowPatch
import scipy
from scipy.interpolate import CubicSpline

fig, ax = plt.subplots(figsize=(10, 10))

ax.scatter(*sim.home_location, marker='*', label='Home', c='red', s=150, zorder=5)
ax.scatter(*sim.bar_location, marker='*', label='Bar', c='orange', s=150, zorder=5)
ax.scatter(*start, marker='*', label='Start', c='black', s=150, zorder=5)

x = np.r_[walk[:, 0]]
y = np.r_[walk[:, 1]]
f, u = scipy.interpolate.splprep([x, y], s=0, per=False)
xint, yint = scipy.interpolate.splev(np.linspace(0, 1, 50000), f)
ax.scatter(x, y, label='Visited intermediates')

points = np.stack([xint, yint]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
coll = LineCollection(segments, cmap='cool', linestyle='dotted')
coll.set_array(np.linspace(0, 1, num=len(points), endpoint=True))
coll.set_linewidth(2)
ax.add_collection(coll)

ax.set_xticks(np.arange(10))
ax.set_yticks(np.arange(10))
ax.set_xlabel('coordinate x')
ax.set_ylabel('coordinate y')
ax.grid()
ax.legend()

In [ ]:
plt.matshow(sim.msm.transition_matrix)

In [ ]:
flux = sim.msm.reactive_flux([sim.coordinate_to_state(start)], [sim.bar_state, sim.home_state])

In [ ]:
net_flux = flux.net_flux.m

In [ ]:
height, width = sim.grid_size

In [ ]:
from scipy import ndimage

In [ ]:
ff = ndimage.gaussian_filter(net_flux, sigma=5, order=1, mode='wrap')

In [ ]:
Q = sim.msm.committor_forward([sim.coordinate_to_state(start)], 
                              [sim.home_state, sim.bar_state]).reshape(sim.grid_size)

In [ ]:
import sktime
import numpy as np
sim = sktime.data.drunkards_walk(bar_location=(0, 0), home_location=(9, 9))
walk = sim.walk(start=start, n_steps=250, seed=17)

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import FancyArrowPatch
import scipy
from scipy.interpolate import CubicSpline

fig, ax = plt.subplots(figsize=(10, 10))

ax.scatter(*sim.home_location, marker='*', label='Home', c='red', s=150, zorder=5)
ax.scatter(*sim.bar_location, marker='*', label='Bar', c='orange', s=150, zorder=5)
ax.scatter(*start, marker='*', label='Start', c='black', s=150, zorder=5)

x = np.r_[walk[:, 0]]
y = np.r_[walk[:, 1]]
f, u = scipy.interpolate.splprep([x, y], s=0, per=False)
xint, yint = scipy.interpolate.splev(np.linspace(0, 1, 50000), f)
ax.scatter(x, y, label='Visited intermediates')
ax.plot(xint, yint, color='black')

ax.imshow(Q, interpolation='nearest', origin='lower')

ax.set_xticks(np.arange(10))
ax.set_yticks(np.arange(10))
ax.set_xlabel('coordinate x')
ax.set_ylabel('coordinate y')
ax.grid()
ax.legend()
# ax.invert_yaxis()

In [ ]:
X = []
Y = []
U = []
V = []
for i in range(0,height-1):
    for j in range(0,width-1):
        X.append(j)
        Y.append(i)
        state_i = sim.coordinate_to_state((i, j))
        f_east = net_flux[i*width+j, i*width+j+1]
        f_south = net_flux[i*width+j, (i+1)*width+j]
        f_west = net_flux[i*width+j, i*width+j-1]
        f_north =  net_flux[i*width+j, (i-1)*width+j]
        U.append(f_east-f_west)
        V.append(f_north-f_south)

C = np.linalg.norm(np.vstack((U,V)),axis=0)        
plt.figure(figsize=(10, 10))
plt.quiver(X, Y, U, V, C)
# plt.imshow(ff)
# plt.gca().invert_yaxis()
plt.gca().set_aspect('equal')    